# How to save and Load lightFM model   
Complete procedure to store and retrieve  local machine model from jupyter notebook to S3 account 


# Steps:  
1. Open Amazon web services account.
2. Do the needful settings.
3. Create object .
4. Develop pickle file (binary format) of machine learning model.
5. Create bucket in S3 and
6. Put that  pickled model object into bucket with proper naming.
7. Get that model and recommend the suitable products to users.  


## 1. Open amazon web service account:
1. Go to website https://aws.amazon.com/, 
2. Click on middle position button having placeholder as ‘Create free account’.
3. For more support visit below link 
https://aws.amazon.com/premiumsupport/knowledge-center/create-and-activate-aws-account/
4. In this way create professional account.


## 2. Do the console settings.  
1. Rover mouse on top-right button named as your user name.
2. Then click on drop down option ‘My Security Credentials’ 
3. Then at the middle of page, click on ‘Access keys (access key ID and secret access key)‘.
4. Click on button named ‘Create New Access key’.
5. In this way we will get two credentials ACCESS_ID and ACCESS_KEY.
6. We can save it at local as .csv file and keep it confidential. 
7. In this way we are able to connect S3 programmatically. 


## 3. Create S3 object in jupyter notebook.  
This will connects local jupyter notebook with aws account using necessary credentials.


In [28]:
!pip install boto3

In [1]:
# Here I have hide my credentials!
ACCESS_ID='XXXXXXXXXXXXXXXXXXX'
ACCESS_KEY='XXXXX+03a75yYzlXXXXXXXXn+eRVFLxIyXXXXXXXXXX'
import boto3
s3 = boto3.resource('s3',
     aws_access_key_id=ACCESS_ID,
     aws_secret_access_key= ACCESS_KEY)

## 4. Pickle the model object.


In [38]:
def model_(df1, subject):
    df = df1.copy()
    from lightfm.data import Dataset
    dataset = Dataset()
    dataset.fit(users=df[subject].unique(), items=df.ProductId.unique())
    (interactions, weights) = dataset.build_interactions([(x[subject], x['ProductId'], x['ratings'])
                                                          for index, x in df.iterrows()])
    
    from lightfm import LightFM
    model = LightFM(loss='warp')
    model.fit(interactions, sample_weight=weights)
    return model

def id_mappings(user_list, item_list,customer_list):
    '''Make the list of dictionary and map with indices to get used'''
    dict_lst =[]
    for lst in [user_list,item_list,customer_list]:
        dict_lst.append({_id:_index for _index, _id in enumerate(lst)})
        dict_lst.append({_index:_id for _index, _id in enumerate(lst)})
        
    return dict_lst[0], dict_lst[1], dict_lst[2], dict_lst[3],dict_lst[4]


def recommendation_dataframe(to_recommend, model, subject, products):
        (recommend_count, name_target) = (50, "userTargetProducts") if subject == 'UserId' \
            else (200, "customerTargetProducts")
        usr_dict_pdoct = {}
        for user in to_recommend:
            import numpy as np
            RecScore = model.predict(user_ids=user, item_ids=products)
            top_items = products[np.argsort(-RecScore)][:recommend_count]
    #             top_items = [productId_dict_Guid[p] for p in top_items]
            usr_dict_pdoct.update({user: list(top_items)})

        df_recommend = pd.DataFrame(list(usr_dict_pdoct.items()), columns=[subject, name_target])
        return df_recommend

In [11]:
@time_to_run
def ratings(df1, subject):
    counting = "UserId" if subject=='CustomerId' else 'CustomerId'
    import swifter
    df = df1.copy()
    df = df[[subject, 'ProductId', 'ProductQuantity',counting]]
    df_grp = df.groupby([subject, 'ProductId']).agg({counting: 'count', 'ProductQuantity': 'sum'})
    df_grp = df_grp.reset_index().sort_values('ProductQuantity', ascending=False)
    df_grp = df_grp.rename(columns={counting: 'NoOfPurchase'})
    QuantMean = df_grp['ProductQuantity'].mean()

    def rating(row1, row2):
        A = (1 <= row1 <= 2)
        B = (3 <= row1 <= 6)
        C = (7 <= row1)
        X = (3 < row2 <= QuantMean)
        Y = (QuantMean < row2)
        Z = row2 <= 3
        return 1 if Z else (2 if A and X else (3 if A and Y else (4 if B and X else
                                                                  (5 if B and Y else (6 if C and X else 7)))))

    df_grp['ratings'] = df_grp.swifter.progress_bar(False).apply(
        lambda row: rating(row.NoOfPurchase, row.ProductQuantity), axis=1) if df_grp.shape[0] > 2000 else df_grp.apply(
        lambda row: rating(row.NoOfPurchase, row.ProductQuantity), axis=1)
#     df_grp['viewCount'] =  np.random.choice([1, 0], df_grp.shape[0])
    return df_grp[[subject, 'ratings', 'ProductId']]

### Here, we will get order data as a interaction matrix for lightFM model.

In [13]:
dframe_model = orderDataRaw[dict_list['recommend']]

In [15]:
dframe_model.head()

,ProductQuantity,UserId,ProductId,CustomerId
0,18,0,0,0
1,45,1,1,1
2,55,2,2,2
3,19,3,3,3
4,66,4,4,4


In [16]:
user, customer = "UserId", "CustomerId"

In [17]:
# For instance, only consider user based model
rating_Frame = ratings(dframe_model, user)

ratings function took 0.648 min


In [21]:
user_model = model_(rating_Frame, user)
#Serialize the model
import pickle
serializedModelObject = pickle.dumps(user_model)

User Featutes None
Item Featutes None


In [23]:
model_name = 'modelUser_{0}'.format(guid_dict_webName[webGuid])

In [24]:
model_name

'modelUser_positive_promotions'

In [31]:
bucket_name= 'model2amazon'

## 5. Create S3 bucket and put the model.


In [ ]:
s3.create_bucket(Bucket=bucket_name)

In [26]:
for bucket in s3.buckets.all():
    print(bucket.name)

model2amazon


In [25]:
# s3.Object(bucket_name, model_name).delete()    # if model already exist and need to re-create it.
s3.Bucket(bucket_name).put_object(Key= model_name, 
                                  Body=serializedModelObject);

s3.Object(bucket_name='model2amazon', key='modelUser_positive_promotions')

## 6. Get the model from S3 bucket


In [32]:
import pickle
my_pickle_model = pickle.loads(s3.Bucket(bucket_name).
                         Object(model_name).get()['Body'].read())

In [36]:
usersId_to_recommend = dframe_model['UserId'].unique()
productsId_to_recommend = dframe_model['ProductId'].unique()
user= "UserId"

In [ ]:
user_Rcommendation = recommendation_dataframe(usersId_to_recommend, my_pickle_model, user, productsId_to_recommend)

In [40]:
user_Rcommendation.head()

,UserId,userTargetProducts
0,0,"[2, 1535, 968, 12621, 2924, 12003, 2437, 1438,..."
1,1,"[2, 1535, 12621, 968, 2924, 1438, 2437, 8383, ..."
2,2,"[2, 1535, 12621, 12003, 2924, 968, 2437, 2262,..."
3,3,"[2, 968, 1535, 12621, 2924, 12003, 2437, 1438,..."
4,4,"[2, 1535, 968, 12621, 2924, 2437, 12003, 8383,..."
